In [38]:
from glob import glob
import xarray as xr
import numpy as np
from parflowio.pyParflowio import PFData

In [39]:
def pfb_arr(fpath):
    pfb = PFData(fpath)
    pfb.loadHeader();
    pfb.loadData();
    arr = pfb.copyDataArray()
    arr = arr.squeeze()
    pfb.close()
    return arr

In [40]:
def wrf_to_da(variable):
    files = sorted(glob(f'wrf_forcings/WRF.{variable}.*.pfb'))
    arr = np.reshape(np.array([pfb_arr(f) for f in files]),(8760,128,64))
    da = xr.DataArray(arr,name=variable,dims=['t','y','x'])
    return da

In [43]:
wrfvars = ['DSWR','DLWR','APCP','Temp','UGRD','VGRD','Press','SPFH']
da_list = [wrf_to_da(v) for v in wrfvars]

In [49]:
ds = xr.Dataset({
    'shortwave':da_list[0],
    'longwave':da_list[1],
    'precip':da_list[2],
    'temp':da_list[3],
    'wind_ew':da_list[4],
    'wind_ns':da_list[5],
    'atm_press':da_list[6],
    'sp_humid':da_list[7],
})

In [50]:
ds.to_netcdf('wrf.nc')

In [51]:
test = xr.open_dataset('wrf.nc')